In [1]:
import numpy as np
from scipy.special import expit as activation_function
from scipy.stats import truncnorm

In [2]:
def truncated_normal(mean=0, sd=1, low=0, upp=10):
  return truncnorm((low - mean) / sd,(upp - mean) / sd,loc=mean,scale=sd)


In [4]:
class NeuralNetwork:

  def __init__(self,network_structure,learning_rate,bias=None):
    self.structure = network_structure
    self.learning_rate = learning_rate
    self.bias = bias
    self.create_weight_matrices()

  def create_weight_matrices(self):
    bias_node = 1 if self.bias else 0
    self.weights_matrices = []
    layer_index = 1
    no_of_layers = len(self.structure)
    while layer_index < no_of_layers:
      nodes_in = self.structure[layer_index-1]
      nodes_out = self.structure[layer_index]
      n = (nodes_in + bias_node) * nodes_out
      rad = 1 / np.sqrt(nodes_in)
      X = truncated_normal(mean=2,sd=1,low=-rad,upp=rad)
      wm = X.rvs(n).reshape((nodes_out, nodes_in + bias_node))
      self.weights_matrices.append(wm)
      layer_index += 1

  def train(self, input_vector, target_vector):
    no_of_layers = len(self.structure)
    input_vector = np.array(input_vector, ndmin=2).T
    layer_index = 0
    res_vectors = [input_vector]
    while layer_index < no_of_layers - 1:
      in_vector = res_vectors[-1]
      if self.bias:
        in_vector = np.concatenate( (in_vector,[[self.bias]]) )
        res_vectors[-1] = in_vector
      x = np.dot(self.weights_matrices[layer_index],in_vector)
      out_vector = activation_function(x)
      res_vectors.append(out_vector)
      layer_index += 1

    layer_index = no_of_layers - 1
    target_vector = np.array(target_vector, ndmin=2).T
    output_errors = target_vector - out_vector
    while layer_index > 0:
      out_vector = res_vectors[layer_index]
      in_vector = res_vectors[layer_index-1]
      if self.bias and not layer_index==(no_of_layers-1):
        out_vector = out_vector[:-1,:].copy()
      tmp = output_errors * out_vector * (1.0 - out_vector)
      tmp = np.dot(tmp, in_vector.T)

      self.weights_matrices[layer_index-1] += self.learning_rate * tmp
      output_errors = np.dot(self.weights_matrices[layer_index-1].T,output_errors)
      if self.bias:
        output_errors = output_errors[:-1,:]
      layer_index -= 1

  def run(self, input_vector):
    no_of_layers = len(self.structure)
    if self.bias:
      input_vector = np.concatenate( (input_vector,[self.bias]) )
    in_vector = np.array(input_vector, ndmin=2).T
    layer_index = 1

    while layer_index < no_of_layers:
      x = np.dot(self.weights_matrices[layer_index-1],in_vector)
      out_vector = activation_function(x)
      in_vector = out_vector
      if self.bias:
        in_vector = np.concatenate( (in_vector,[[self.bias]]))
      layer_index += 1
    return out_vector

    def evaluate(self, data, labels):
      corrects, wrongs = 0, 0
      for i in range(len(data)):
        res = self.run(data[i])
        res_max = res.argmax()
        if res_max == labels[i]:
          corrects += 1
        else:
          wrongs += 1
      return corrects, wrongs


